In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder \
    .master("local[*]")\
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

In [3]:
from pipeline_oriented_analytics import Phase
phase = Phase.predict

data_to_predict_path = f'../data/processed/{phase.name}/features'
model_path = '../model/trip_duration_min'
predicted_data_path = '../data/reporting/trip_durations'

In [6]:
model = PipelineModel.load(model_path)

predicted_df = PipelineModel([
    model,
    DropColumns(inputCols=['features']),
    SaveToParquet(predicted_data_path)
]).transform(ParquetDataFrame(data_to_predict_path, spark))

predicted_df.show(2)
spark.stop()

+--------------+-------------+---------+--------+-----+------------+-----------+----+--------------------+---------------------+-----------------+------------------+------------------+
|dropoff_cell_8|pickup_cell_8|       id|distance|month|day_of_month|day_of_week|hour|requests_pickup_cell|requests_dropoff_cell|pickup_cell_8_idx|dropoff_cell_8_idx|        prediction|
+--------------+-------------+---------+--------+-----+------------+-----------+----+--------------------+---------------------+-----------------+------------------+------------------+
|         89c25|        89c25|id2463079|    3.25|    1|           4|          2|  13|                  31|                   31|              0.0|               0.0|15.574324324324325|
|         89c25|        89c25|id0750307|    4.17|    1|           4|          2|  13|                  31|                   31|              0.0|               0.0|13.483056900246012|
+--------------+-------------+---------+--------+-----+------------+-------

In [7]:
predicted_df.select('pickup_cell_6_idx').groupby('pickup_cell_6_idx').count().show()

+-----------------+------+
|pickup_cell_6_idx| count|
+-----------------+------+
|              0.0|625112|
|              7.0|     1|
|              1.0|     7|
|             11.0|     1|
|              2.0|     6|
|              6.0|     1|
|              5.0|     1|
|             26.0|     4|
|             16.0|     1|
+-----------------+------+

